<a href="https://colab.research.google.com/github/byunmj24/MyStudy/blob/master/practice3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
View = pd.read_csv('ab_test_view.csv')
Click = pd.read_csv('ab_test_click.csv')

In [3]:
View.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87924 entries, 0 to 87923
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   log_date        87924 non-null  object
 1   app_name        87924 non-null  object
 2   test_name       87924 non-null  object
 3   test_case       87924 non-null  object
 4   user_id         87924 non-null  int64 
 5   transaction_id  87924 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 4.0+ MB


In [4]:
Click.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8598 entries, 0 to 8597
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   log_date        8598 non-null   object
 1   app_name        8598 non-null   object
 2   test_name       8598 non-null   object
 3   test_case       8598 non-null   object
 4   user_id         8598 non-null   int64 
 5   transaction_id  8598 non-null   int64 
dtypes: int64(2), object(4)
memory usage: 403.2+ KB


In [5]:
# 데이터 전처리
# 각 데이터들을 결합

ViewClick = View.merge(Click, on=['transaction_id'], how='outer')
ViewClick.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87927 entries, 0 to 87926
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   log_date_x      87924 non-null  object 
 1   app_name_x      87924 non-null  object 
 2   test_name_x     87924 non-null  object 
 3   test_case_x     87924 non-null  object 
 4   user_id_x       87924 non-null  float64
 5   transaction_id  87927 non-null  int64  
 6   log_date_y      8598 non-null   object 
 7   app_name_y      8598 non-null   object 
 8   test_name_y     8598 non-null   object 
 9   test_case_y     8598 non-null   object 
 10  user_id_y       8598 non-null   float64
dtypes: float64(2), int64(1), object(8)
memory usage: 8.0+ MB


In [6]:
ViewClick[:5]

,log_date_x,app_name_x,test_name_x,test_case_x,user_id_x,transaction_id,log_date_y,app_name_y,test_name_y,test_case_y,user_id_y
0,2019-10-01,game-01,sales_test,B,36703.0,25622,NaN,NaN,NaN,NaN,NaN
1,2019-10-01,game-01,sales_test,A,44339.0,25623,NaN,NaN,NaN,NaN,NaN
2,2019-10-01,game-01,sales_test,B,32087.0,25624,NaN,NaN,NaN,NaN,NaN
3,2019-10-01,game-01,sales_test,B,10160.0,25625,NaN,NaN,NaN,NaN,NaN
4,2019-10-01,game-01,sales_test,B,46113.0,25626,NaN,NaN,NaN,NaN,NaN


In [7]:
# 일자별 정렬하기 위해 날짜 처리
ViewClick['log_date_x'] = ViewClick['log_date_x'].apply(lambda x:pd.to_datetime(str(x)))

In [8]:
ViewClick.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87927 entries, 0 to 87926
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   log_date_x      87924 non-null  datetime64[ns]
 1   app_name_x      87924 non-null  object        
 2   test_name_x     87924 non-null  object        
 3   test_case_x     87924 non-null  object        
 4   user_id_x       87924 non-null  float64       
 5   transaction_id  87927 non-null  int64         
 6   log_date_y      8598 non-null   object        
 7   app_name_y      8598 non-null   object        
 8   test_name_y     8598 non-null   object        
 9   test_case_y     8598 non-null   object        
 10  user_id_y       8598 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 8.0+ MB


In [9]:
# 광고별로 클릭률에 대한 지표를 설정
# 데이터프레임 Click에서 User_id 값이 존재하면 "광고를 클릭"한 것임
# ViewClick 데이터프레임에 "is_click" 열을 만들고, "User_id_y"값이 True이면 0, 그렇지 않으면 1로 표시
ViewClick["is_click"] = np.where(pd.isnull(ViewClick['user_id_y'])==True, 0, 1)

In [10]:
ViewClick[1003:]

,log_date_x,app_name_x,test_name_x,test_case_x,user_id_x,transaction_id,log_date_y,app_name_y,test_name_y,test_case_y,user_id_y,is_click
1003,2019-10-01,game-01,sales_test,A,18315.0,26625,2019-10-01,game-01,sales_test,A,18315.0,1
1004,2019-10-01,game-01,sales_test,B,15191.0,26626,NaN,NaN,NaN,NaN,NaN,0
1005,2019-10-01,game-01,sales_test,B,41082.0,26627,NaN,NaN,NaN,NaN,NaN,0
1006,2019-10-01,game-01,sales_test,A,49608.0,26628,NaN,NaN,NaN,NaN,NaN,0
1007,2019-10-01,game-01,sales_test,A,43412.0,26629,2019-10-01,game-01,sales_test,A,43412.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
87922,2019-10-31,game-01,sales_test,B,57610.0,85314,2019-10-31,game-01,sales_test,B,57610.0,1
87923,2019-10-31,game-01,sales_test,B,56445.0,85315,NaN,NaN,NaN,NaN,NaN,0
87924,NaT,NaN,NaN,NaN,NaN,12013,2019-10-16,game-01,sales_test,B,53400.0,1
87925,NaT,NaN,NaN,NaN,NaN,72013,2019-10-19,game-01,sales_test,B,52316.0,1


In [11]:
# 데이터 분석
# 1) 광고별 클릭률 확인
ViewClick['is_click'].groupby(ViewClick['test_case_x']).mean()

# 광고 A는 클릭률  8%(0.080256)을 보이고 있음
# 광고 B는 클릭률 11%(0.115392)를 보이고 있음
# 따라서, 광고 B가 더 나은 결과를 보이고 있음

test_case_x
A    0.080256
B    0.115392
Name: is_click, dtype: float64